In [ ]:
!pip install SpeechRecognition

In [ ]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.4.0/en_ner_bc5cdr_md-0.4.0.tar.gz

In [ ]:
!pip install pydub

In [ ]:
!pip install rasa_nlu

In [ ]:
!pip install rasa_core

In [ ]:
pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-2.3.1/en_core_web_sm-2.3.1.tar.gz

In [ ]:
!pip install sklearn_crfsuite

In [9]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import requests
import json

In [ ]:
print('-------Enter the link for the data to be tested--------')
L=input()

-------Enter the link for the data to be tested--------
http://api.open-notify.org/iss-now.json


In [10]:
cd '/content/drive/MyDrive/SpeechRecognition/'

/content/drive/MyDrive/SpeechRecognition


In [ ]:
request = requests.get(L)
D=request.json()

In [ ]:
import os,sys
from pydub import AudioSegment
def con():
  orig_song = D["audio"]
  dest_song = os.path.splitext(D["audio"])[0]+'.wav'
  song = AudioSegment.from_ogg(orig_song)
  song.export(dest_song, format="wav")
  
  return dest_song


In [ ]:
import speech_recognition as sr
def speech_text(audio):
  r = sr.Recognizer()
  with sr.AudioFile(audio) as source:
    audio_text = r.listen(source)
  try:
    text = r.recognize_google(audio_text)
  except:
    print('Sorry.. run again...')
  return text

In [ ]:
audio=con()
text=speech_text(audio)

In [ ]:
import rasa_nlu
import rasa_core
import spacy
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config
training_data = load_data("q1/nlu1.md")
trainer = Trainer(config.load("config.yml"))
interpreter = trainer.train(training_data)
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")
def q11(text):
  x=interpreter.parse(text)
  val=[]
  for i in x['entities']:
    val.append(i['value'])
  return val


In [ ]:
import en_ner_bc5cdr_md
nlp_bc = en_ner_bc5cdr_md.load()
def q1(text):
  doc=nlp_bc(text)
  return doc.ents

In [ ]:
import re
def q2(text,options):
  temp = re.findall(r'\d+', text)
  res = list(map(int, temp))
  for i in options:
    r=i
    i=i.replace('lakh','')
    if '>' in i :
      i=i.replace('>','')
      if res[0] > int(i):
        return r
    elif '<' in i :
      i=i.replace('<','')
      if res[0] < int(i):
        return r
    else:
      i=i.split('-')
      if res[0]>=int(i[0]) and res[0]<=int(i[1]):
        return r

In [ ]:
def convert(text):
  month_to_number = {
  'january' : 1,         
  'february' : 2,         
  'march' : 3,           
  'april' : 4,              
  'may' : 5, 
  'june' : 6,
  'july' : 7, 
  'august' : 8, 
  'september' : 9, 
  'october' : 10, 
  'november' : 11, 
  'december' : 12}
  s=text
  for i in s.split():
    if i in month_to_number.keys():
      x=[v for k, v in month_to_number.items() if i.lower() in k.lower()][0]
      return x

In [ ]:
def q3(text):
  temp=re.findall(r'\d+',text)
  res=list(map(int,temp))
  if len(res)==3:
    return res
  else:
    l=[]
    mon=convert(text)
    l.append(res[0])
    l.append(mon)
    l.append(res[1])
    return l
  

In [ ]:
q=D['question_key']
if q=='q1':
  options=D['options']
  answers2=q11(text)
  answers=str(q1(text))
  if (len(answers)>2):
    answers=answers.replace(",","")
    answers=answers.replace("(","")
    answers=answers.replace(")","")
    answers=answers.split(" ")
    answers=list(set(answers) or set(answers2))
    answers=list(set(answers) and set(options))
  else :
    l_s=['none','nothing','no']
    text=text.split(" ")
    for i in text:
      if i in l_s:
        answers=i
  x={"answers":answers}
elif q=='q2':
  options=D['options']
  answers=q2(text,options)
  x={"answers":[answers]}
else:
  l=q3(text)
  answers=str(l[0])+'/'+str(l[1])+'/'+str(l[2])
  x={"answers":[answers]}


print(x)